In [ ]:
 #기본 모듈
import numpy as np
import pandas as pd
from tqdm import tqdm

#전처리 모듈
from sklearn.preprocessing import RobustScaler

#Pytorch
import torch
import torch.nn as nn

## Model Load

#### 1.모델 클래스 정의

In [ ]:
#LSTM 모델 클래스 정의(summer)
class LSTM_summer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(LSTM_summer, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# 하이퍼파라미터 설정
input_size_s = 15
hidden_size_ = 100  # 은닉 상태 차원(100,150)
learning_rate = 0.0005 # 학습률(0.0005)
num_layers = 2  # LSTM 층의 수
output_size = 1  # 출력 차원
num_epochs = 100  # 학습 횟수
batch_size = 64  # 배치 크기
save_interval = 10  # 모델을 저장할 epoch 간격
dropout_prob = 0.3  # 드롭아웃 비율
save_dir = '../MODELS'  # 모델 저장 경로
early_stopping_patience = 10  # 조기 종료를 위한 인내 횟수

In [ ]:
#LSTM 모델 클래스 정의(non_summer)
class LSTMmodel_non_summer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(LSTMmodel_non_summer, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

#하이퍼파라미터 설정
input_size = 14
hidden_size = 100  # 은닉 상태 차원(100,150)
learning_rate = 0.0005 # 학습률(0.0005)
num_layers = 2  # LSTM 층의 수
output_size = 1  # 출력 차원
num_epochs = 100  # 학습 횟수
batch_size = 64  # 배치 크기
save_interval = 10  # 모델을 저장할 epoch 간격
dropout_prob = 0.3  # 드롭아웃 비율
save_dir = '../MODELS'  # 모델 저장 경로
early_stopping_patience = 10  # 조기 종료를 위한 인내 횟수

#### 2.모델 객체 생성

In [ ]:
# 모델 생성 및 디바이스 설정
LSTMmodel_summer = LSTM_summer(input_size_s, hidden_size, num_layers, output_size, dropout_prob) 
LSTMmodel_non_summer = LSTMmodel_non_summer(input_size, hidden_size, num_layers, output_size, dropout_prob) 

#모델 저장 경로
load_path_summer = '../MODELS/summer_hidden100_lr0.0005/LSTM_summer.pth'
load_path_non_summer = '../MODELS/non_summer_hidden100_lr0.0005/LSTM_non_summer.pth'

# GPU 사용 여부 확인 및 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# summer model load
LSTMmodel_summer.load_state_dict(torch.load(load_path_summer, map_location=device))
LSTMmodel_summer
LSTMmodel_summer.eval()  # 모델을 평가 모드로 전환

# non_summer model load
LSTMmodel_non_summer.load_state_dict(torch.load(load_path_non_summer, map_location=device))
LSTMmodel_non_summer
LSTMmodel_non_summer.eval()  # 모델을 평가 모드로 전환

print(f'Model loaded from {load_path_summer}')
print(f'Model loaded from {load_path_non_summer}')

## Preprocessing Function

#### 1.시간 파생변수

In [ ]:
def preprocess_time(data):
    data['TM'] = pd.to_datetime(data['TM']) #datetime 형식으로 변환

    #시간 순으로 데이터 정렬
    data = data.sort_values(by='TM')

    data['YY'] = data['TM'].dt.year #년을 새로운 열로 추가
    data['MM'] = data['TM'].dt.month #월을 새로운 열로 추가
    data['DD'] = data['TM'].dt.day #일을 새로운 열로 추가
    return data

#### 2.결측치 처리하기

In [ ]:
def fillnan(data):
    #결측치(-99)를 nan으로 수정
    data.replace(-99, np.nan, inplace=True) 

    #시계열 데이터를 인덱스로 지정
    data.set_index('TM', inplace=True)

    #선형보간법으로 결측치 처리
    data.interpolate(method='time', inplace=True)

    #인덱스 되돌리기
    data.reset_index(drop=True, inplace=True)

    return data

#### 3.계절 레이블 생성

In [ ]:
def create_season_label(data):
    input = data.copy()

    #7월과 8월일 때 1, 그렇지 않을 때 0로 계절 레이블 생성
    input['season_label'] = input['MM'].isin([7, 8]).astype(int)

    return input

#### 4.MAVG Decomposition

In [ ]:
def moving_average_decomposition(data, window=24):
    input = data.copy()

    # 결과를 저장할 열 추가
    input['trend_s'] = 0.0
    input['seasonal_s'] = 0.0
    input['residual_s'] = 0.0

    # 고유한 시즌 라벨에 대해 반복
    for label in input['season_label'].unique():
        subset = input[input['season_label'] == label].copy()
        
        #이동 평균을 이용한 추세 추정
        subset['trend_s'] = subset['elect'].rolling(window=window, center=True).mean()
        subset['trend_s'] = subset['trend_s'].ffill()
        subset['trend_s'] = subset['trend_s'].bfill()  #앞뒤 NaN 값 채우기
        
        #계절성 추정 (원래 값에서 추세를 뺀 값)
        subset['seasonal_s'] = subset['elect'] - subset['trend_s']
        subset['seasonal_s'] = subset['seasonal_s'].ffill()
        subset['seasonal_s'] = subset['seasonal_s'].bfill()  #앞뒤 NaN 값 채우기
        
        #잔차 계산 (원래 값에서 추세와 계절성을 뺀 값)
        subset['residual_s'] = subset['elect'] - subset['trend_s'] - subset['seasonal_s']
        subset['residual_s'] = subset['residual_s'].ffill()
        subset['residual_s'] = subset['residual_s'].bfill()  #앞뒤 NaN 값 채우기
        
        #원래 데이터 프레임에 결과 할당
        input.loc[subset.index, 'trend_s'] = subset['trend_s']
        input.loc[subset.index, 'seasonal_s'] = subset['seasonal_s']
        input.loc[subset.index, 'residual_s'] = subset['residual_s']

    return input

#### 5.차분값 파생변수

In [ ]:
def diff_data(data):
    input = data.copy()

    # 전력 차분
    input['difference1_e'] = input['elect'].diff()  
    input['difference2_e'] = input['elect'].diff(2)  
    input['difference3_e'] = input['elect'].diff(3) 

    # 습도 차분
    input['difference1_h'] = input['nph_hm'].diff()  
    input['difference2_h'] = input['nph_hm'].diff(2)  
    input['difference3_h'] = input['nph_hm'].diff(3)  

    # 기온 차분
    input['difference1_t'] = input['nph_ta'].diff() 
    input['difference2_t'] = input['nph_ta'].diff(2) 
    input['difference3_t'] = input['nph_ta'].diff(3) 

    # 앞쪽 채움 처리 후 nan 값이 남아있을 경우 뒤쪽 채움으로 처리
    input['difference1_e'] = input['difference1_e'].ffill().bfill()
    input['difference2_e'] = input['difference2_e'].ffill().bfill()
    input['difference3_e'] = input['difference3_e'].ffill().bfill()

    input['difference1_h'] = input['difference1_h'].ffill().bfill()
    input['difference2_h'] = input['difference2_h'].ffill().bfill()
    input['difference3_h'] = input['difference3_h'].ffill().bfill()
    
    input['difference1_t'] = input['difference1_t'].ffill().bfill()
    input['difference2_t'] = input['difference2_t'].ffill().bfill()
    input['difference3_t'] = input['difference3_t'].ffill().bfill()

    return input

#### 6.Scailing

In [ ]:
#scailing
def scailing_data(data, selected_features):
    input = data.copy()

    #스케일러 초기화
    scaler_R = RobustScaler()

    #스케일링이 필요한 컬럼 목록
    S_columns = ['NUM', 'YY', 'MM', 'DD', 'HH24', 'weekday', 'week_name', 'STN',
                'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi', 'elect']


    #스케일링하는 데이터와 하지 않는 컬럼 분리
    scaling_features = [feature for feature in selected_features if feature in S_columns]
    non_scaling_features = [feature for feature in selected_features if feature not in S_columns]

    #필요한 데이터만 추출
    scale_data = input[scaling_features].copy()
    non_scale_data = input[non_scaling_features].copy()

    #스케일러 적용
    scaled_R = scaler_R.fit_transform(scale_data)

    #결과를 다시 DataFrame으로 변환
    scaled_R = pd.DataFrame(scaled_R, columns=scale_data.columns)

    final_data = pd.concat([scaled_R, non_scale_data.reset_index(drop=True)], axis=1)

    return final_data

#### 7.시퀀스 생성

In [ ]:
def create_sequence(data):
    input = data.copy()

    seq_length=24
    num_sequences = len(input) - seq_length + 1

    #Numpy 배열을 미리 할당하여 메모리 사용을 최적화
    vs = np.zeros((num_sequences, seq_length, input.shape[1]))
    
    for i in range(num_sequences):
        vs[i] = input.iloc[i:i+seq_length].values

    #가장 마지막 24시간 시퀀스를 선택
    recent_sequence = vs[-1] 
    
    return recent_sequence

#### 8.텐서로 변환

In [ ]:
def toTensor(vs):
    #입력값이 ndarray인지 확인
    if not isinstance(vs, np.ndarray):
        raise ValueError("Input data must be a numpy ndarray")
    
    # 데이터 복사를 피하고, float32로 변환
    tensor = torch.from_numpy(vs).float().unsqueeze(0)  # 배치 차원 추가

    return tensor

### Predict Function

In [ ]:
def predict(model, input):
    input = input.to(device)  # 입력 데이터를 GPU로 이동
    model = model.to(device)  # 모델을 GPU로 이동
    with torch.no_grad():  # 예측 시에는 역전파를 사용하지 않도록 설정
        prediction = model(input)
    return prediction.cpu().numpy().flatten()[0]

### Process Function

In [ ]:
def process(data, start_date, end_date, pred_time):
    #시간 파생변수
    time_data = preprocess_time(data)

    #3개월치 데이터 자르기
    mask = (time_data['TM'] >= start_date) & (time_data['TM'] <= end_date)
    filtered_data = time_data.loc[mask].copy()

    #예측할 시간의 기상데이터(예측 데이터)
    pred_data = data.loc[data['TM'] == pred_time].copy()

    #모델과 피처값 선정하기
    pred_data = preprocess_time(pred_data)
    if (pred_data['MM'].isin([7, 8])).any():
        model = LSTMmodel_summer
        features = ['difference3_t', 'MM', 'nph_ta', 'DD', 'difference1_e', 'week_name', 'trend_s', 'difference2_e',
                    'difference3_h', 'HH24', 'difference2_h', 'seasonal_s', 'nph_ta_chi', 'residual_s', 'elect']
        
    else:
        model = LSTMmodel_non_summer
        features = ['difference3_t', 'MM', 'nph_ta', 'DD', 'week_name', 'trend_s', 'difference2_e',
                    'difference3_h', 'HH24', 'difference2_h', 'seasonal_s', 'nph_ta_chi', 'residual_s', 'elect']

    #결측치 처리
    fillnan_data = fillnan(filtered_data)

    #계절 레이블 생성
    seasonal_data = create_season_label(fillnan_data)

    #MAVG decomposition
    mavg_data = moving_average_decomposition(seasonal_data)

    #차분값 파생변수
    diffed_data = diff_data(mavg_data)

    #데이터 스케일링
    scaled_data = scailing_data(diffed_data, features)

    #sequence 생성
    sequence_data = create_sequence(scaled_data)

    #tensor 변환
    tensor_data = toTensor(sequence_data)

    #예측하기
    prediction = predict(model, tensor_data)

    return prediction


## Main

In [ ]:
#데이터 로드(2022년 3개월 데이터를 2023년 데이터와 합침)
data = pd.read_csv("../Data/combined_data.csv")

In [ ]:
def main():
    #시작 날짜와 끝 날짜 설정
    start_date = '2022-10-01 01:00:00'
    end_date  = '2024-01-01 00:00:00'

    #1시간 단위로 날짜 범위 생성
    times = pd.date_range(start=start_date, end=end_date, freq='h')

    #시작 시간과 끝 시간을 3개월 간격으로 유지
    for i in tqdm(range(len(times) - 1), desc="Processing time intervals"):
        current_start_date = times[i].strftime('%Y-%m-%d %H:%M:%S')
        current_end_date = (pd.to_datetime(current_start_date) + pd.DateOffset(months=3) - pd.Timedelta(hours=1)).strftime('%Y-%m-%d %H:%M:%S')
        
        if i % 200 == 0:
            print(f'Processing from {current_start_date} to {current_end_date}')
        
        pred_time = (pd.to_datetime(current_end_date) + pd.Timedelta(hours=1)).strftime('%Y-%m-%d %H:%M:%S')
        prediction = process(data, current_start_date, current_end_date, pred_time) #예측값을 prediction에 저장
        prediction = np.float64(prediction).round(2)  #float64로 변환 및 소수점 2자리로 반올림
        data.loc[data['TM'] == pred_time, 'elect'] = prediction #prediction을 data에 저장
        
    print('Processing complete.')

In [ ]:
main()

In [ ]:
#필요한 데이터만 잘라내기
pred_data = data[(data['TM'] >= '2023-01-01 01:00:00') & (data['TM'] <= '2024-01-01 00:00:00')]

In [ ]:
pred_data.head()

In [ ]:
#csv 파일로 저장하기
pred_data.to_csv('240409.csv', index=False)